# Cleaning

## Imports

In [1]:
import pandas as pd
import json
import sys
import os

sys.path.append(os.path.abspath('../src'))

import api.spotify.search as searcher

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [7]:
base = pd.read_csv('../data/Most_Streamed_Spotify_Songs_2024_utf8.csv')
base.head()

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


## Corrupted Characters

The creator of the CSV dataset improperly encoded the data when uploading, resulting in many corrupted characters, e.g.,

1. `Track` on Line **89**: Titï¿½ï¿½ Me Pregu
2. `Album Name` on Line **28**: ýýýýýýýýý ýýýýýý ýýýýýýýýýýýý

After unsuccessfully trying various decoding methods, I turned to Spotify's API to fix the corrupted text. Each track in the dataset has an ISRC (International Standard Recording Code), and Spotify provides ISRC metadata for most tracks. By matching ISRCs, I retrieve the correct metadata and replace the corrupted text with properly encoded UTF-8 values.

In [3]:
base_corrupt = base[['Track', 'Album Name', 'Artist', 'ISRC']]

In [14]:
base_corrupt[30:42]

,Track,Album Name,Artist,ISRC
30,Danza Kuduro - Cover,ýýýýýýýýýýýýýýýýýýýýýýýýýýý ýýýýýýýýýýýýýýýýýý...,LOVE BGM JPN,TCJPA2406007
31,SHEESH,BABYMONS7ER,BABYMONSTER,KRA402400016
32,Water,Water,Tyla,USSM12305126
33,Strangers,Strangers,Kenya Grace,USWB12305048
34,Too Sweet,Unreal Unearth: Unheard,Hozier,USSM12401865
35,What's Love Got to Do with It - 2023 Remaster,What's Love Got to Do with It (30th Anniversar...,Tina Turner,GBAYE2300351
36,A Bar Song (Tipsy),A Bar Song (Tipsy),Shaboozey,USUYG1541249
37,Lose Control,Lose Control,Teddy Swims,USWB12302315
38,fukumean,fukumean,Gunna,USAT22306575
39,Dance Monkey,Dance Monkey,Tones And I,QZES71982312


### Using the Spotify API to correct corrupted rows

All the code for Spotify API authentication and search functionalities is located in `src/api/spotify/` under the project root directory. Here, we find the corrupted rows in the dataset, and use the Spotify API to search for the correct values. The results are stored in accepted and rejected dataframes, where accepted holds all corrected rows (as well rows that were correct to begin with), and rejected holds rows that could not be corrected.

In [15]:
import pandas as pd
import time

def contains_non_ascii(s):
    """Check if string contains any non-ASCII characters."""
    return any(ord(c) >= 128 for c in str(s))

def corrupted_row(row):
    """Check if a row contains any non-ASCII strings."""
    return any(contains_non_ascii(row[col]) for col in ['Track', 'Album Name', 'Artist'])

def count_corrupted_rows(df):
    """Count corrupted rows."""
    return sum(corrupted_row(row) for _, row in df.iterrows())

def clean_corrupted_rows(df):
    """Correct corrupted rows."""
    accepted, rejected = [], []
    
    i = 0
    total_corrupted_rows = count_corrupted_rows(df)
    print('Corrupted rows processed:')
    
    for _, row in df.iterrows():
        if corrupted_row(row):
            result = searcher.search_isrc(row['ISRC'])
            time.sleep(0.5) # adhere to rate limit (conservatively ~120 req/min)
            if result:
                accepted.append(result)
            else:
                rejected.append(row.to_dict())
            i += 1
            print(f'{i}/{total_corrupted_rows} --- {round((i/total_corrupted_rows)*100)}% complete.')
        else:
            accepted.append(row.to_dict())
            
    return pd.DataFrame(accepted), pd.DataFrame(rejected)

accepted, rejected = clean_corrupted_rows(base_corrupt)

Corrupted rows processed:
1/595 --- 0% complete.
2/595 --- 0% complete.
3/595 --- 1% complete.
4/595 --- 1% complete.


2025-02-01 20:19:20,221 - WARNING - No results found for ISRC QZNJW2423715


5/595 --- 1% complete.
6/595 --- 1% complete.


2025-02-01 20:19:21,482 - WARNING - No results found for ISRC NZAM02001128


7/595 --- 1% complete.
8/595 --- 1% complete.
9/595 --- 2% complete.
10/595 --- 2% complete.
11/595 --- 2% complete.
12/595 --- 2% complete.
13/595 --- 2% complete.
14/595 --- 2% complete.
15/595 --- 3% complete.
16/595 --- 3% complete.
17/595 --- 3% complete.
18/595 --- 3% complete.
19/595 --- 3% complete.
20/595 --- 3% complete.
21/595 --- 4% complete.
22/595 --- 4% complete.
23/595 --- 4% complete.
24/595 --- 4% complete.
25/595 --- 4% complete.
26/595 --- 4% complete.
27/595 --- 5% complete.
28/595 --- 5% complete.
29/595 --- 5% complete.
30/595 --- 5% complete.
31/595 --- 5% complete.
32/595 --- 5% complete.
33/595 --- 6% complete.
34/595 --- 6% complete.
35/595 --- 6% complete.
36/595 --- 6% complete.
37/595 --- 6% complete.
38/595 --- 6% complete.
39/595 --- 7% complete.
40/595 --- 7% complete.
41/595 --- 7% complete.
42/595 --- 7% complete.


...

527/595 --- 89% complete.
528/595 --- 89% complete.
529/595 --- 89% complete.
530/595 --- 89% complete.
531/595 --- 89% complete.
532/595 --- 89% complete.
533/595 --- 90% complete.
534/595 --- 90% complete.
535/595 --- 90% complete.
536/595 --- 90% complete.
537/595 --- 90% complete.
538/595 --- 90% complete.
539/595 --- 91% complete.
540/595 --- 91% complete.
541/595 --- 91% complete.
542/595 --- 91% complete.
543/595 --- 91% complete.
544/595 --- 91% complete.
545/595 --- 92% complete.
546/595 --- 92% complete.
547/595 --- 92% complete.
548/595 --- 92% complete.
549/595 --- 92% complete.
550/595 --- 92% complete.
551/595 --- 93% complete.
552/595 --- 93% complete.
553/595 --- 93% complete.
554/595 --- 93% complete.
555/595 --- 93% complete.
556/595 --- 93% complete.
557/595 --- 94% complete.
558/595 --- 94% complete.
559/595 --- 94% complete.
560/595 --- 94% complete.
561/595 --- 94% complete.
562/595 --- 94% complete.
563/595 --- 95% complete.
564/595 --- 95% complete.
565/595 --- 

2025-02-01 20:25:43,328 - WARNING - No results found for ISRC FR0Z50062905


588/595 --- 99% complete.
589/595 --- 99% complete.


2025-02-01 20:25:44,553 - WARNING - No results found for ISRC FR10S1809158


590/595 --- 99% complete.
591/595 --- 99% complete.
592/595 --- 99% complete.
593/595 --- 100% complete.
594/595 --- 100% complete.
595/595 --- 100% complete.


There are only 22 corrupted rows that could not be corrected. Given the number of rows the original dataset, and the fact that keeping these rows with faulty ISRCs would be problematic for further API calls, I exclude them from the dataset.

In [18]:
rejected.head()

,Track,Album Name,Artist,ISRC
0,Dona Felicidade,Dona Felicidade,Patati Patatï¿,QZNJW2423715
1,Laxed ï¿½ï¿½ï¿½ Sire,Laxed (Siren Beat),Jawsh 685,NZAM02001128
2,I DONï¿½ï¿½ï¿½T WANNA DO THIS A,King of the Dead,Juliï¿½ï¿½n Kh,QZLL92480334
3,Cuando Te Amï¿,Cuando Te Amï¿½ï¿½ - Si,Cupula Roja,UKXN22459926
4,Lean On (feat. Mï¿½ï¿½ & DJ Snake) - J Balvin ...,Lean On (feat. Mï¿½ï¿½ & DJ Snake) [J Balvin &...,Major Lazer,FR6P11501180


In [22]:
len(rejected)

22

### Joining the base dataframe and accepted dataframe

Now the accepted dataframe and base dataframe can be joined. To do this, an inner merge is appropriate, because accepted has less rows than the base dataframe, and we want to drop rows that are in the rejected dataframe. 

In [27]:
accepted.head()

,Track,Album Name,Artist,ISRC
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,QM24S2402528
1,Not Like Us,Not Like Us,Kendrick Lamar,USUG12400910
2,i like the way you kiss me,I like the way you kiss me,Artemas,QZJ842400387
3,Flowers,Flowers - Single,Miley Cyrus,USSM12209777
4,Houdini,Houdini,Eminem,USUG12403398


In [29]:
# drop info columns from base dataframe for easy merge
base_to_merge = base.drop(columns=['Track', 'Album Name', 'Artist'])
base_to_merge.head()

,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",92.0,"84,274,754","1,713,126",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",92.0,"116,347,040","3,486,739",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",92.0,"122,599,116","2,228,730",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",85.0,"1,096,100,899","10,629,796",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",88.0,"77,373,957","3,670,188",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


We should expect to get `len(accepted) = 4578` rows after merging.

In [47]:
print(f'{len(base_to_merge)} - {len(rejected)} = {len(accepted)}')

4600 - 22 = 4578


In [34]:
# inner merge accepted and base_to_merge dataframes
base_not_corrupt = base_to_merge.merge(accepted, on='ISRC', how='inner')
base_not_corrupt.head()

,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,...,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track,Track,Album Name,Artist
0,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",92.0,"84,274,754","1,713,126",...,114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman
1,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",92.0,"116,347,040","3,486,739",...,111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1,Not Like Us,Not Like Us,Kendrick Lamar
2,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",92.0,"122,599,116","2,228,730",...,172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0,i like the way you kiss me,I like the way you kiss me,Artemas
3,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",85.0,"1,096,100,899","10,629,796",...,210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0,Flowers,Flowers - Single,Miley Cyrus
4,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",88.0,"77,373,957","3,670,188",...,105.0,"4,493,884","7,006","207,179","457,017",NaN,1,Houdini,Houdini,Eminem


In [41]:
len(base_not_corrupt)

4582

### Removing duplicate rows

We get 4 additional rows. Why? Lets check for duplicated rows:

In [40]:
base_not_corrupt[base_not_corrupt['ISRC'].duplicated()]

,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,...,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track,Track,Album Name,Artist
2441,9/2/2022,TCAGJ2289254,"2,424",28.9,"227,893,586","28,139","12,480,714",73.0,"30,913,276","159,791",...,49.0,"56,972,562","26,968","1,336,043","708,143",NaN,0,Tennessee Orange,Tennessee Orange,Megan Moroney
2442,9/2/2022,TCAGJ2289254,"2,424",28.9,"227,893,586","28,139","12,480,714",73.0,"30,913,276","159,791",...,49.0,"56,972,562","26,968","1,336,043","708,143",NaN,0,Tennessee Orange,Tennessee Orange,Megan Moroney
2443,9/2/2022,TCAGJ2289254,"2,424",28.9,"227,893,586","28,139","12,480,714",73.0,"30,913,276","159,791",...,49.0,"56,972,562","26,968","1,336,043","708,143",NaN,0,Tennessee Orange,Tennessee Orange,Megan Moroney
3433,12/8/2017,USWL11700269,"3,441",23.3,"579,189,526","60,397","11,805,084",65.0,"828,853,696","3,086,157",...,12.0,"6,723,858","7,832",NaN,"1,619,550",NaN,0,Dembow,Dembow,Danny Ocean
3434,12/8/2017,USWL11700269,"3,441",23.3,"579,189,526","60,397","11,805,084",65.0,"828,853,696","3,086,157",...,12.0,"6,723,858","7,832",NaN,"1,619,550",NaN,0,Dembow,Dembow,Danny Ocean
3435,12/8/2017,USWL11700269,"3,441",23.3,"579,189,526","60,397","11,805,084",65.0,"828,853,696","3,086,157",...,12.0,"6,723,858","7,832",NaN,"1,619,550",NaN,0,Dembow,Dembow,Danny Ocean


We can see we have 4 duplicated rows. Lets remove these:

In [48]:
base_not_corrupt = base_not_corrupt.drop_duplicates(subset=['ISRC'], keep='first')
len(base_not_corrupt)

4576

We lost an additional 2 rows because the accepted dataframe already had duplicates. We are now duplicate free:

In [52]:
base_not_corrupt['ISRC'].duplicated().any()

False

### Reordering the columns

Finally, lets move `Track`, `Album Name`, and `Artist` back to the first beginning of the dataframe, along with `ISRC`.

In [53]:
col_to_move = ['ISRC', 'Track', 'Album Name', 'Artist']

col_order = col_to_move + [col for col in base_not_corrupt.columns if col not in col_to_move]

base_not_corrupt = base_not_corrupt[col_order]
base_not_corrupt.head()

,ISRC,Track,Album Name,Artist,Release Date,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,QM24S2402528,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,USUG12400910,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,QZJ842400387,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,USSM12209777,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,USUG12403398,Houdini,Houdini,Eminem,5/31/2024,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


### Saving the non-corrupted dataset

In [54]:
base_not_corrupt.to_csv('../data/base.csv', index=False, encoding='utf-8')